In [0]:
import sys

import sys
sys.path.append("/Workspace/Users/jessepepple36@gmail.com/End-to-End-Sales-Data-Engineering-Pipeline-Azure-Databricks-DLT-CI-CD-/dab_salesproject/prod/files/utils/Silver_Transformations.py")


from utilities.Silver_Transformations import Transformations

In [0]:
class Transformations:
    def __init__(self, df=None):
        self.df = df

    def drop_cols(self, *col):
        for col in col:
            self.df = self.df.drop(col)
        return self.df

    def read_Streams(self, data_name, folder_name):
        self.df = spark.readStream.format("cloudFiles") \
            .option("cloudFiles.format", "csv") \
            .option("cloudFiles.InferColumnTypes", "true") \
            .option("cloudFiles.schemaLocation", f"abfss://silver@trsdatalakejess.dfs.core.windows.net/{folder_name}/checkpoint_location") \
            .option("schemaEvolutionMode", "addNewColumns") \
            .load(f"abfss://bronze@trsdatalakejess.dfs.core.windows.net/{data_name}")
        return self.df

    def writeStream(self, data_name, folder_name):
        self.df.writeStream.format("delta") \
            .outputMode("append") \
            .option("checkpointLocation", f"abfss://silver@trsdatalakejess.dfs.core.windows.net/{folder_name}/checkpoint_location") \
            .option("path", f"abfss://silver@trsdatalakejess.dfs.core.windows.net/{folder_name}/{data_name}") \
            .trigger(once=True) \
            .toTable(f"sales_project.silver.{data_name}")
        return self.df

    def drop_data(self, *col_name):
        for c in col_name:
            self.df = self.df.drop(c)
        return self.df

    def rename_cols(self, old_name: str, new_name: str):
        self.df = self.df.withColumnRenamed(old_name, new_name)
        return self.df

    def drop_duplicates(self, col_name):
        self.df = self.df.dropDuplicates(col_name)
        return self.df
    
    def cast_Int(self, col_names: list):
        for col_name in col_names:
            self.df = self.df.withColumn(col_name, self.df[col_name].cast(IntegerType()))
        return self.df

    def add_updatedtimestamp(self, column_name):
        self.df = self.df.withColumn(column_name, current_timestamp())
        return self.df


In [0]:
%sql
DROP TABLE sales_project.silver.dim_customer

## Customer Ingestion And Transformations With AutoLoaders

In [0]:
Customer_Clean = Transformations()

In [0]:

df_customer = Customer_Clean.read_Streams("Dim_Customer", "dim_customer_enr")


df_customer.display()


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df_customer = Customer_Clean.drop_cols("_rescued_data")
df_customer.display()

In [0]:
df_customer = Customer_Clean.rename_cols("customer_code", "customer_id")
df_customer.display

In [0]:

df_customer = Customer_Clean.add_updatedtimestamp("last_updated")
df_customer.display()
    


In [0]:
df_customer = Customer_Clean.drop_duplicates(["customer_id"])
df_customer.display()

In [0]:
Customer_Clean.writeStream("dim_customer", "dim_customer_enr")


## Dim_Product Ingestion And Transformations With AutoLoaders

In [0]:
Clean_Products = Transformations(df_products)

In [0]:
df_products = Clean_Products.read_Streams("Dim_product", "dim_products_enr")
df_products.display()

In [0]:
df_products = Clean_Products.drop_cols("_rescued_data", "uom")
df_products = Clean_Products.add_updatedtimestamp("last_updated")
df_products = Clean_Products.rename_cols("product_code", "product_id")
df_products = Clean_Products.drop_duplicates(["product_id"])
df_products.display()

In [0]:
Clean_Products.writeStream("dim_products", "dim_products_enr")

## Dim_Store Ingestion And Transformations With AutoLoaders

In [0]:
Clean_Stores = Transformations()


In [0]:
df_stores = Clean_Stores.read_Streams("Dim_Store", "dim_stores_enr")

In [0]:
df_stores = Clean_Stores.drop_cols("_rescued_data")
df_stores = Clean_Stores.add_updatedtimestamp("last_updated")
df_stores = Clean_Stores.rename_cols("store_code", "store_id")
df_stores = Clean_Stores.drop_duplicates(["store_id"])
df_stores.display()

In [0]:
df_stores = df_stores.withColumn("region", regexp_replace("region", "Northeast", "NorthEast"))

df_stores.display()

In [0]:
Clean_Stores.writeStream("dim_stores", "dim_stores_enr")

## Dim_Date Ingestion And Transformations With AutoLoaders

In [0]:
Clean_Date = Transformations()

In [0]:
df_date = Clean_Date.read_Streams("Dim_Date", "dim_date_enr")


In [0]:
df_date.display()

In [0]:
df_date = Clean_Date.drop_cols("_rescued_data")
df_date = Clean_Date.add_updatedtimestamp("last_updated")
df_date = Clean_Date.add_updatedtimestamp("last_updated")
df_date = Clean_Date.drop_duplicates(["date_sk"])
df_date.display()

In [0]:
Clean_Date.writeStream("dim_date", "dim_date_enr")

## FactSales Ingestion And Transformations With AutoLoaders

In [0]:
Clean_FactsSales = Transformations()

In [0]:
df_facts = Clean_FactsSales.read_Streams("Fact_Sales", "fact_sales_enr")


In [0]:
df_facts.display()

In [0]:
df_facts = Clean_FactsSales.drop_cols("_rescued_data", "promotion_sk", "payment_method")
df_facts = Clean_FactsSales.add_updatedtimestamp("last_updated")
df_facts = Clean_FactsSales.drop_duplicates(["sales_id"])
df_facts.display()

In [0]:
Clean_FactsSales.writeStream("fact_sales", "fact_sales_enr")
